In [1]:
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

In [4]:
import transformers
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)

Transformers version: 4.38.1
Torch version: 2.1.0+cu121


In [8]:
sentences = [
    "The dog barked loudly, startling the sleeping cat nearby.",
    "Sarah rushed to catch the bus, her backpack bouncing against her back with each hurried step.",
    "The smell of freshly baked bread wafted from the bakery, tempting passersby with its warm aroma.",
    "James scrolled through his phone, his thumb flicking rapidly as he browsed through social media updates.",
    "The old man sat quietly on the park bench, feeding breadcrumbs to the pigeons that gathered around him.",
    "Emily giggled as she watched the colorful balloons float into the sky, carried away by the gentle breeze."
]


In [6]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # tokenize sentence and append to dictionary lists
    new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True,
                                       padding='max_length', return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [10]:
tokens

{'input_ids': [tensor([  101,  1996,  3899, 17554,  9928,  1010, 19828,  1996,  5777,  4937,
           3518,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [15]:
type(tokens['attention_mask']),type(tokens['attention_mask'][0]), type(tokens)

(list, torch.Tensor, dict)

In [16]:
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [17]:
type(tokens['attention_mask']),type(tokens['attention_mask'][0]), type(tokens)

(torch.Tensor, torch.Tensor, dict)

In [32]:
tokens

{'input_ids': tensor([[  101,  1996,  3899, 17554,  9928,  1010, 19828,  1996,  5777,  4937,
           3518,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

In [20]:
tokens['input_ids'].shape, tokens['attention_mask'].shape

(torch.Size([6, 128]), torch.Size([6, 128]))

In [21]:
outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [22]:
embeddings = outputs.last_hidden_state
embeddings.shape #It contains embeddings of padded words we need to get rid of

torch.Size([6, 128, 768])

In [23]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([6, 128])

In [24]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([6, 128, 768])

In [25]:
masked_embeddings = embeddings * mask
masked_embeddings.shape #masked embeddings now contains 0 in case of padded words

torch.Size([6, 128, 768])

In [26]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([6, 768])

In [27]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([6, 768])

In [28]:
mean_pooled = summed / summed_mask

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.33945894, 0.25457034, 0.21069369, 0.26938823, 0.31310707]],
      dtype=float32)

In [35]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.9 MB/s eta 0:00:00


In [37]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [38]:
sentence_embeddings = model.encode(sentences)

In [39]:
sentence_embeddings.shape

(6, 768)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

array([[0.33945894, 0.25457034, 0.21069369, 0.26938823, 0.31310707]],
      dtype=float32)

In [ ]:
#https://www.sbert.net/docs/training/overview.html  : Bert Models in SBERT Documentation